In [1]:
import pandas as pd
import numpy as np
import os
import sys
from nilearn.glm import expression_to_contrast_vector
sys.path.append('..')
from utils.analysis import est_c_vifs, est_vifs, est_efficiency

In [2]:
first_lvl_dir = '/Users/hugofluhr/phd_local/data/LearningHabits/spm_outputs/glm1_2025-06-05-05-15'

In [3]:
def load_dms(first_lvl_dir):
    # Get list of subject directories
    subjects = [d for d in os.listdir(first_lvl_dir) if d.startswith('sub-')]
    subjects.sort()

    DMs = {}
    # walk through each subject and run to load design matrices
    for sub in subjects:
        DMs[sub] = {}
        for run in [f'run-{i}' for i in range(1, 4)]:
            design_matrix_path = os.path.join(first_lvl_dir, sub, run, f'{sub}_{run}_design_matrix.csv')
            column_names_path = os.path.join(first_lvl_dir, sub, run, f'{sub}_{run}_column_names.txt')
            if os.path.exists(design_matrix_path) and os.path.exists(column_names_path):
                with open(column_names_path, 'r') as f:
                    col_names = [line.strip() for line in f]
                DMs[sub][run] = pd.read_csv(design_matrix_path, names=col_names, header=None)
    return DMs

In [4]:
DMs = load_dms(first_lvl_dir)

In [5]:
def cVIFs_summary(DMs, n_regs = 7):
    sample_dm = next(iter(next(iter(DMs.values())).values()))
    contrasts = {col:col for col in sample_dm.columns[:n_regs]}
    cVIFs = []
    for _, runs in DMs.items():
        for _, dm in runs.items():
            cVIFs.append(est_c_vifs(dm, contrasts))
    runs = [f'run-{i}' for i in range(1, 4)]
    subjects = list(DMs.keys())
    index = [(sub, run) for sub in subjects for run in runs]
    df_vifs = pd.DataFrame(cVIFs, index=pd.MultiIndex.from_tuples(index, names=['subject', 'run']))
    return df_vifs.describe()

In [10]:
def eff_summary(DMs, n_regs = 7):
    sample_dm = next(iter(next(iter(DMs.values())).values()))
    contrasts = {col:col for col in sample_dm.columns[:n_regs]}
    effs = []
    for _, runs in DMs.items():
        for _, dm in runs.items():
            effs.append(est_efficiency(dm, contrasts))
    runs = [f'run-{i}' for i in range(1, 4)]
    subjects = list(DMs.keys())
    index = [(sub, run) for sub in subjects for run in runs]
    df_effs = pd.DataFrame(effs, index=pd.MultiIndex.from_tuples(index, names=['subject', 'run']))
    return df_effs.describe()

In [6]:
def VIFs_summary(DMs, n_regs = 7):
    sample_dm = next(iter(next(iter(DMs.values())).values()))
    regressors = sample_dm.columns[:n_regs]
    VIFs = []
    for _, runs in DMs.items():
        for _, dm in runs.items():
            VIFs.append(est_vifs(dm, regressors))
    runs = [f'run-{i}' for i in range(1, 4)]
    subjects = list(DMs.keys())
    index = [(sub, run) for sub in subjects for run in runs]
    df_vifs = pd.DataFrame(VIFs, index=pd.MultiIndex.from_tuples(index, names=['subject', 'run']))
    return df_vifs.describe()

In [7]:
cVIFs_summary(DMs)

,Sn(1) first_stim*bf(1),Sn(1) first_stimxQval^1*bf(1),Sn(1) first_stimxHval^1*bf(1),Sn(1) second_stim*bf(1),Sn(1) response*bf(1),Sn(1) feedback*bf(1),Sn(1) first_stim_excl*bf(1)
count,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000
mean,16.597477,1.178005,1.328539,11.886019,49.627241,13.280107,10.337450
std,3.358924,0.102918,0.197152,2.769490,5.812404,1.276761,2.499572
min,10.053973,1.063698,1.051599,5.913146,36.963293,10.624233,4.547384
25%,14.173684,1.111447,1.178490,10.030425,45.867095,12.318561,8.632922
50%,16.020681,1.156574,1.274923,11.834031,49.252511,13.144555,9.885517
75%,18.552227,1.210517,1.434213,13.461572,52.932901,14.160415,11.937641
max,26.930413,1.777338,2.176355,25.540812,70.388134,17.981682,19.091960


In [8]:
VIFs_summary(DMs)

,Sn(1) first_stim*bf(1),Sn(1) first_stimxQval^1*bf(1),Sn(1) first_stimxHval^1*bf(1),Sn(1) second_stim*bf(1),Sn(1) response*bf(1),Sn(1) feedback*bf(1),Sn(1) first_stim_excl*bf(1)
count,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000
mean,16.597477,1.178005,1.328539,11.886019,49.627241,13.280107,10.337450
std,3.358924,0.102918,0.197152,2.769490,5.812404,1.276761,2.499572
min,10.053973,1.063698,1.051599,5.913146,36.963293,10.624233,4.547384
25%,14.173684,1.111447,1.178490,10.030425,45.867095,12.318561,8.632922
50%,16.020681,1.156574,1.274923,11.834031,49.252511,13.144555,9.885517
75%,18.552227,1.210517,1.434213,13.461572,52.932901,14.160415,11.937641
max,26.930413,1.777338,2.176355,25.540812,70.388134,17.981682,19.091960


In [11]:
eff_summary(DMs)

,Sn(1) first_stim*bf(1),Sn(1) first_stimxQval^1*bf(1),Sn(1) first_stimxHval^1*bf(1),Sn(1) second_stim*bf(1),Sn(1) response*bf(1),Sn(1) feedback*bf(1),Sn(1) first_stim_excl*bf(1)
count,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000
mean,0.062551,0.854306,0.767067,0.088737,0.020421,0.075975,0.102331
std,0.011797,0.063179,0.099209,0.021224,0.002357,0.007112,0.024718
min,0.037133,0.562639,0.459484,0.039153,0.014207,0.055612,0.052378
25%,0.053902,0.826093,0.697247,0.074286,0.018892,0.070619,0.083769
50%,0.062420,0.864622,0.784362,0.084502,0.020304,0.076077,0.101160
75%,0.070553,0.899728,0.848546,0.099697,0.021802,0.081178,0.115836
max,0.099463,0.940117,0.950932,0.169115,0.027054,0.094124,0.219907
